In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from pathlib import Path
from scipy import stats as scipy_stats
from statsmodels.stats.multitest import multipletests

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Open two-start-two-choice-results.csv as a pandas dataframe
data_path = Path.home() / 'VS Code Local' / 'corner-maze-analysis' / 'data' / 'processed' / 'two-start-two-choice-results.parquet'
tstc_df = pd.read_parquet(data_path)
# set panda display options
pd.set_option('display.max_columns', None,
              'display.max_rows', None,
              'display.expand_frame_repr', False)

In [ ]:
# Create summary dataframe to test ANOVA assumptions training types on trials to acquisition
# will include sex factor, but approach can not be tested here since PI training has no cue.
summary_df = tstc_df[tstc_df['training_type'].isin(['PI', 'PI+VC_f2', 'VC'])].copy()
# Only include pure behavior rats no neuro manipulation or imaging
summary_df = summary_df[summary_df['name'].str.contains(r'CM')]
trials_to_acq_full_df = summary_df[['sex', 'cue_approach', 'training_type', 'trials_to_acq']].copy()
print(trials_to_acq_full_df)
DV = 'trials_to_acq'
FACTORS_FULL = ['sex', 'cue_approach', 'training_type']

def _levene_groups_column(df: pd.DataFrame, factors: list[str] | None=None) -> pd.Series:
    # Create a grouping column for Levene's test based on factors
    return df[factors].astype(str).agg('_'.join, axis=1)


def check_assumptions(df: pd.DataFrame,
                      factors: list[str] | None=None,
                      dv: str = None
                      ) -> dict:
    from scipy import stats
    from statsmodels.stats.diagnostic import lilliefors
    # Basic DV checks
    desc = df[dv].describe()
    skew = stats.skew(df[dv], nan_policy="omit")
    kurt = stats.kurtosis(df[dv], fisher=True, nan_policy="omit")

    # OLS residuals for normality (fit saturated ANOVA model)
    import statsmodels.api as sm
    from statsmodels.formula.api import ols
    formula = f'{dv} ~ ' + " * ".join([f'C({f})' for f in factors])
    model = ols(formula, data=df).fit()
    resid = model.resid.dropna()

    # Shapiro (n<=5000 recommended) and Lilliefors (Kolmogorov–Smirnov adj)
    shapiro_p = np.nan
    if len(resid) <= 5000:
        shapiro_p = stats.shapiro(resid)[1]
    try:
        lillie_stat, lillie_p = lilliefors(resid.values, dist='norm')
    except Exception:
        lillie_p = np.nan

    # Levene across cells
    from scipy.stats import levene
    grp = _levene_groups_column(df, factors)
    groups = [df.loc[grp==g, dv].dropna().values for g in sorted(grp.unique())]
    # filter empty groups
    groups = [g for g in groups if len(g)>1]
    lev = levene(*groups, center='median') if len(groups)>=2 else None

    return {
        "dv_describe": desc.to_dict(),
        "skew": float(skew) if np.isfinite(skew) else np.nan,
        "kurtosis_fisher": float(kurt) if np.isfinite(kurt) else np.nan,
        "resid_shapiro_p": float(shapiro_p) if shapiro_p==shapiro_p else np.nan,
        "resid_lilliefors_p": float(lillie_p) if lillie_p==lillie_p else np.nan,
        "levene_W": float(lev.statistic) if lev else np.nan,
        "levene_p": float(lev.pvalue) if lev else np.nan,
        "n": int(df[dv].notna().sum())
    }

print("Assumptions Check - Full SCT (with PI and Cue Approach):")
full_assumptions = check_assumptions(trials_to_acq_full_df, FACTORS_FULL, DV)
for assump, value in full_assumptions.items():
    print(f"{assump}: {value}")


fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=trials_to_acq_full_df, x=DV, bins='auto', color='steelblue',
             edgecolor='black', alpha=0.8, kde=False, ax=ax)
ax.set_title('Distribution of trials_to_acq')
ax.set_xlabel('Trials to Acquisition')
ax.set_ylabel('Count')
plt.tight_layout()
plt.show()

Empty DataFrame
Columns: [sex, cue_approach, training_type, trials_to_acq]
Index: []
Assumptions Check - Full SCT (with PI and Cue Approach):


/var/folders/qt/zv138w997kq5m4sbp80ysg1h0000gn/T/ipykernel_46927/3068552603.py:24: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  skew = stats.skew(df[dv], nan_policy="omit")
/var/folders/qt/zv138w997kq5m4sbp80ysg1h0000gn/T/ipykernel_46927/3068552603.py:25: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  kurt = stats.kurtosis(df[dv], fisher=True, nan_policy="omit")


ValueError: negative dimensions are not allowed